In [6]:
import torch
import os
import json
import utils
import model.net as net  # Ensure this matches your model definition
import numpy as np
import pandas as pd
import torch
from scipy import stats
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from scipy import stats
from model import net
import utils
import matplotlib
# matplotlib.use("TkAgg")  # Use a GUI-compatible backend
matplotlib.use('Agg')  # replace TkAgg as Agg

In [24]:
# Load trained model
current_dir = os.getcwd()

model_dir = os.path.join("experiments", "base_model")
params = utils.Params(os.path.join(model_dir, "params.json"))
params.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = net.Net(params).to(params.device)
utils.load_checkpoint(os.path.join(model_dir, "epoch_3.pth.tar"), model) # test a bunch of model
model.eval()

# Load dataset
data_path = os.path.join("..", "data", "stock", "cvs_stock_wsenti.csv") #  load data here
data = pd.read_csv(data_path, parse_dates=['Date'])
data['Date'] = pd.to_datetime(data['Date'])
data.set_index("Date", inplace=True)


forecast_start = "2025-02-18" # date started to predict
train_window = 30
num_covariates = 9
future_steps = 5  #predict number of days

last_30_days = data.loc[:forecast_start].iloc[-train_window:]
last_30_days.fillna(method='ffill', inplace=True)
# price_data = last_30_days[['High', 'Low', 'Open', 'Close', 'Volume', 'Daily Return']]
price_data = last_30_days[['High', 'Low', 'Open', 'Close', 'Volume', 'Sentiment_Score', 'Daily Return']]

# Generate covariates
def gen_covariates(times, price_data, num_covariates):
    covariates = np.zeros((len(times), num_covariates))
    covariates[:, 0] = stats.zscore([t.weekday() for t in times])
    covariates[:, 1] = stats.zscore([t.month for t in times])
    covariates[:, 2] = stats.zscore(price_data['Close'].shift(5).values)
    covariates[:, 3] = stats.zscore(price_data['Volume'].shift(5).values)
    intraday_return = (price_data['Close'] - price_data['Open']) / price_data['Open']
    covariates[:, 4] = stats.zscore(intraday_return.shift(5).values)
    ma5 = price_data['Close'].rolling(window=5).mean()
    covariates[:, 5] = stats.zscore((price_data['Close'] - ma5).values)
    exp1 = price_data['Close'].ewm(span=12, adjust=False).mean()
    exp2 = price_data['Close'].ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    covariates[:, 6] = stats.zscore(macd.shift(2).values)
    volatility = (price_data['High'] - price_data['Low']) / price_data['Close']
    covariates[:, 7] = stats.zscore(volatility.values)
    covariates[:, 8] = stats.zscore(price_data['Sentiment_Score'].shift(5).values)
    return np.nan_to_num(covariates)

covariates = gen_covariates(last_30_days.index, price_data, num_covariates)

# Prepare input tensor
x_input = np.zeros((1, train_window, 1 + num_covariates), dtype='float32')
x_input[0, 1:, 0] = last_30_days['Daily Return'].values[1:]
x_input[0, :, 1:1 + num_covariates] = covariates[-train_window:, :]
new_input_tensor = torch.tensor(x_input, dtype=torch.float32).permute(1, 0, 2).to(params.device)

# Predict future steps
batch_size = new_input_tensor.shape[1]
hidden = model.init_hidden(batch_size)
cell = model.init_cell(batch_size)
idx = torch.zeros(1, batch_size, dtype=torch.long, device=params.device)
predictions = []

for _ in range(future_steps):
    mu, sigma, hidden, cell = model(new_input_tensor[-1].unsqueeze_(0), idx, hidden, cell)
    next_value = mu.cpu().detach().numpy().squeeze()
    predictions.append(next_value)
    new_input = np.roll(new_input_tensor.cpu().numpy(), shift=-1, axis=0)
    new_input[-1, 0, 0] = next_value
    new_input_tensor = torch.tensor(new_input, dtype=torch.float32).to(params.device)

# Convert returns to stock prices
last_price = last_30_days['Close'].iloc[-1]
predicted_prices = [last_price]
for ret in predictions:
    next_price = predicted_prices[-1] * (1 + ret)
    predicted_prices.append(next_price)
predicted_prices = np.array(predicted_prices[1:])

# Plot predictions
future_dates = pd.date_range(start=forecast_start, periods=future_steps)
plt.figure(figsize=(12, 6))
plt.plot(future_dates, predicted_prices, marker='o', linestyle='-', label="Predicted Prices")
plt.axvline(x=pd.to_datetime(forecast_start), color='red', linestyle='--', label='Prediction Start')
plt.title("Predicted Stock Prices")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.xticks(rotation=45)
plt.legend()
plt.grid()

figures_dir = os.path.join(model_dir, "figures")
os.makedirs(figures_dir, exist_ok=True)
plot_path = os.path.join(figures_dir, "cvs_predict.png")
plt.savefig(plot_path)
print(f"Plot saved to {plot_path}")

/home/yic075/CAPSTONE-stockreturn/DeepAR/utils.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint, map_location='cpu')
/tmp/ipykernel_3

Plot saved to experiments/base_model/figures/cvs_predict_best.png


# real price

In [26]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
current_dir = os.getcwd()
capstone_dir = os.path.join(os.path.expanduser("~"), "CAPSTONE-stockreturn")
file_path = os.path.join(capstone_dir, "data", "stock", "cvs_stock_wsenti.csv")
data = pd.read_csv(file_path, parse_dates=["Date"])

# Filter the data for the given date range
start_date = "2025-02-18"
end_date = "2025-02-24"
filtered_data = data[(data["Date"] >= start_date) & (data["Date"] <= end_date)]

# Plot the Close price
plt.figure(figsize=(10, 5))
plt.plot(filtered_data["Date"], filtered_data["Close"], marker='o', linestyle='-', label="Close Price")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.title(f"CVS Stock Close Price from {start_date} to {end_date}")
plt.xticks(filtered_data["Date"], rotation=45)  # Ensure the labels reflect only the selected date range
plt.legend()
plt.grid()

# Save the figure instead of showing it
figures_dir = os.path.join(capstone_dir, "DeepAR", "experiments", "base_model", "figures")
os.makedirs(figures_dir, exist_ok=True) 
plot_path = os.path.join(figures_dir, "cvs_stock_plot.png")
plt.savefig(plot_path)
print(f"Plot saved to {plot_path}")

Plot saved to /home/yic075/CAPSTONE-stockreturn/DeepAR/experiments/base_model/figures/cvs_stock_plot.png
